In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify


In [ ]:
DATASET_DIR = "food-101/images"
CALORIE_FILE = "calorie_data.csv"

In [ ]:
DATASET_DIR = "food-101/images"
CALORIE_FILE = "calorie_data.csv"

# Step 1: Data Preprocessing
print("Setting up data generators...")
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = data_gen.flow_from_directory(
    DATASET_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = data_gen.flow_from_directory(
    DATASET_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
print("Building the model...")
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Freeze the base model during initial training
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print("Training the model...")
model.fit(train_generator, validation_data=val_generator, epochs=10)

In [ ]:
MODEL_PATH = "food_recognition_model.h5"
model.save(MODEL_PATH)

In [ ]:
print("Loading calorie data...")
calorie_data = pd.read_csv(CALORIE_FILE)
def map_calories(label):
    match = calorie_data[calorie_data['food'] == label]
    return match['calories'].values[0] if not match.empty else None

In [ ]:
def predict_calories(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_label = list(train_generator.class_indices.keys())[np.argmax(predictions[0])]
    calorie_value = map_calories(predicted_label)
    return predicted_label, calorie_value

In [ ]:
print("Setting up the API...")
app = Flask(__name__)

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({"error": "No image file provided"}), 400

    image = request.files['image']
    image_path = os.path.join("uploads", image.filename)
    image.save(image_path)

    label, calories = predict_calories(image_path)
    os.remove(image_path)  # Clean up the uploaded image

    if calories is None:
        return jsonify({"error": "Calorie information not available for the predicted food"}), 404

    return jsonify({"food": label, "calories": calories})

    if __name__ == '__main__':
    os.makedirs("uploads", exist_ok=True)
    app.run(debug=True)
